In [1]:
from __future__ import division
import os, sys, re, json, time, datetime, shutil
import itertools, collections

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf
assert(tf.__version__.startswith("1."))

# Helper libraries
from common import utils, vocabulary, tf_embed_viz, treeviz
from common import patched_numpy_io

In [2]:
bias_sentences = pd.read_csv("bias_sentences.csv")

In [3]:
bias_sentences.count()

sentence    113299
label       113299
dtype: int64

In [4]:
def weak_annotator(sent, list1, list2):
    ind1 = ind2 = len(sent)
    if any(word in sent for word in list1):
        ind1 = max([sent.lower().find(word) for word in list1])
    elif any(word in sent for word in list2):
        ind2 = max([sent.lower().find(word) for word in list2])
    else:
        ind1 = ind2 = len(sent)
        
    if ind1 < ind2:
        return 'liberal'
    elif ind2 < ind1:
        return 'conservative'
    else: 
        return 'neutral'       

In [5]:
lib_list = ['liberal','democrat','pro-choice','agnostic','abortion','gay','freedom'
            ,'climate','secular','civil','liberty','equality','regulation'
            ,'violence','ACLU','right','far-right']

In [6]:
con_list = ['conservative','republican','pro-life','catholic','free market','tax cut','second amendment'
            ,'fair trade','GOP','NRA','christian','left','left-wing']

In [7]:
ibc_database = pd.read_csv("ibcData.csv",sep='\t', header=None)

In [8]:
ibc_database.columns

Int64Index([0, 1], dtype='int64')

In [9]:
ibc_database[1].unique()

array(['liberal', 'conservative', 'neutral'], dtype=object)

In [10]:
weak_labels_ibc = []
 
for i in range(ibc_database.count()[0]):
    weak_labels_ibc.append(weak_annotator(ibc_database.iloc[i][0],lib_list,con_list))

In [12]:
weak_labels_bias = []
 
for i in range(bias_sentences.count()['sentence']):
    weak_labels_bias.append(weak_annotator(bias_sentences.iloc[i]['sentence'],lib_list,con_list))

In [13]:
bias_sentences['label'] = weak_labels_bias

In [14]:
bias_sentences.head()

,sentence,label
0,"In the last 365 days, we've had problems: crit...",neutral
1,"As expected, the end of the quarter and the ye...",neutral
2,This traditional ''window-dressing'' helped sh...,neutral
3,Last year I was spot-on about the collapse of ...,neutral
4,THE average consumer's telephone bill has ris...,neutral


In [16]:
weak_labels_ibc.count('liberal')

762

In [17]:
weak_labels_ibc.count('neutral')

3334

In [18]:
weak_labels_ibc.count('conservative')

230

In [19]:
ibc_database['weak_label'] = weak_labels_ibc

In [20]:
ibc_database.head()

,0,1,weak_label
0,Forcing middle-class workers to bear a greater...,liberal,neutral
1,Because it would not be worthwhile to bring a ...,liberal,neutral
2,"Indeed , Lind argues that high profits and hig...",liberal,neutral
3,"In fairness , it should be noted that he devot...",liberal,liberal
4,Psychological tactics are social control techn...,liberal,neutral


In [21]:
# tokenize IBC database
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

X = ibc_database[0]
U = bias_sentences['sentence']

X_tokens = []
U_tokens = []

for i in range(len(X)):
    X_tokens.append(tokenizer.tokenize(X[i]))
    
for i in range(len(U)):
    U_tokens.append(tokenizer.tokenize(U[i]))

In [22]:
# canonicalize IBC database

X_tokens_canon = []
U_tokens_canon = []

for i in range(len(X_tokens)):
    X_tokens_canon.append(utils.canonicalize_words(X_tokens[i]))
    
for i in range(len(U_tokens)):
    U_tokens_canon.append(utils.canonicalize_words(U_tokens[i]))

In [23]:
# flatten the list of lists consisting of canonicalized sentences

X_final = list(itertools.chain.from_iterable(X_tokens_canon))
U_final = list(itertools.chain.from_iterable(U_tokens_canon))

In [24]:
# number of unique tokens in database
print(len(set(X_final)))
print(len(set(U_final)))

15721
54687


In [25]:
# create unigrams, could be used for creating distribution plots

unigram_counts_X = collections.Counter()
unigram_counts_U = collections.Counter()

for word in X_final:  
    unigram_counts_X[word] += 1
    
for word in U_final:  
    unigram_counts_U[word] += 1

In [26]:
words_X = unigram_counts_X.keys()

counts_X = []
for w in words_X:
    counts_X.append(unigram_counts_X[w])
    
words_U = unigram_counts_U.keys()

counts_U = []
for w in words_U:
    counts_U.append(unigram_counts_U[w])

In [27]:
# sorting the words according to their counts

unigram_list_X = [x for _,x in sorted(zip(counts_X,words_X),  reverse=True)]
unigram_count_X = [y for y,_ in sorted(zip(counts_X,words_X),  reverse=True)]

unigram_list_U = [x for _,x in sorted(zip(counts_U,words_U),  reverse=True)]
unigram_count_U = [y for y,_ in sorted(zip(counts_U,words_U),  reverse=True)]

In [28]:
# plot the distribution of unigrams

utils.require_package("bokeh")
import bokeh.plotting as bp
from bokeh.models import HoverTool
bp.output_notebook()

Loading BokehJS ...

In [29]:
hist, bin_edges = np.histogram(a=unigram_count_X, bins=40, normed=True)

In [30]:
nplot = 100
fig = bp.figure(x_range=unigram_list_X[:nplot], plot_width=1000, plot_height=600)
bars = fig.vbar(x=unigram_list_X[:nplot], width=0.4, top=unigram_count_X[:nplot], hover_fill_color="firebrick")
fig.add_tools(HoverTool(tooltips=[("word", "@x"), ("count", "@top")], renderers=[bars], mode="vline"))
fig.y_range.start = 0
fig.y_range.end = 1.2*max(unigram_count_X)
fig.yaxis.axis_label = "Count(w)"
fig.xgrid.grid_line_alpha = 0.75
fig.xaxis.major_label_orientation = "vertical"
bp.show(fig)

In [31]:
hist, bin_edges = np.histogram(a=unigram_count_U, bins=40, normed=True)

In [32]:
nplot = 100
fig = bp.figure(x_range=unigram_list_U[:nplot], plot_width=1000, plot_height=600)
bars = fig.vbar(x=unigram_list_U[:nplot], width=0.4, top=unigram_count_U[:nplot], hover_fill_color="firebrick")
fig.add_tools(HoverTool(tooltips=[("word", "@x"), ("count", "@top")], renderers=[bars], mode="vline"))
fig.y_range.start = 0
fig.y_range.end = 1.2*max(unigram_count_U)
fig.yaxis.axis_label = "Count(w)"
fig.xgrid.grid_line_alpha = 0.75
fig.xaxis.major_label_orientation = "vertical"
bp.show(fig)

In [33]:
# initially we set vocabulary size equal to total number of unique tokens in the two dataset

mergelist = set(unigram_list_X + unigram_list_U)

V = len(mergelist)

mergelist_XU = X_final + U_final


vocab = vocabulary.Vocabulary(mergelist_XU, size=V)

In [34]:
# find the maximum length of sentences
lengths_X = map(len,X_tokens_canon)
lengths_U = map(len,U_tokens_canon)

In [35]:
lengths_X = list(lengths_X)
lengths_U = list(lengths_U)
max_len1 = max(lengths_X)
max_len2 = max(lengths_U)
max_len = max(max_len1,max_len2)

In [36]:
X_ids = []
U_ids = []
for i in range(len(X_tokens_canon)):
    X_ids.append(vocab.words_to_ids(X_tokens_canon[i]))
    
for i in range(len(U_tokens_canon)):
    U_ids.append(vocab.words_to_ids(U_tokens_canon[i]))

In [37]:
# final padded sentences
X_sent, ns = utils.pad_np_array(X_ids, max_len=max_len1, pad_id=0)
U_sent, u_ns = utils.pad_np_array(U_ids, max_len=max_len2, pad_id=0)

In [38]:
print(X_sent.shape)
print(U_sent.shape)

(4326, 86)
(113299, 493)


In [39]:
Y = ibc_database[1]
Y_u = bias_sentences['label']

In [40]:
Y = list(Y)
Y_u = list(Y_u)

In [41]:
# final labels
Y_label = []

for i in range(len(Y)):
    if Y[i] == 'liberal':
        Y_label.append(0)
    elif Y[i] == 'neutral':
        Y_label.append(1)
    else:
        Y_label.append(2)

Y_u_label = []

for i in range(len(Y_u)):
    if Y_u[i] == 'liberal':
        Y_u_label.append(0)
    elif Y_u[i] == 'neutral':
        Y_u_label.append(1)
    else:
        Y_u_label.append(2)

In [42]:
Y_w = ibc_database['weak_label']

Y_w =list(Y_w)
Y_weak = []
for i in range(len(Y)):
    if Y_w[i] == 'liberal':
        Y_weak.append(0)
    elif Y_w[i] == 'neutral':
        Y_weak.append(1)
    else:
        Y_weak.append(2)

In [43]:
X_sent_w = []
for i in range(X_sent.shape[0]):
    X_sent_w.append(np.append(X_sent[i], [Y_weak[i],ns[i]]))

U_sent_w = []
for i in range(U_sent.shape[0]):
    U_sent_w.append(np.append(U_sent[i], [u_ns[i]]))

In [44]:
X_sent_w = np.reshape(X_sent_w, newshape= (X_sent.shape[0],X_sent.shape[1]+2))
U_sent_w = np.reshape(U_sent_w, newshape= (U_sent.shape[0],U_sent.shape[1]+1))

In [45]:
len(Y_u_label)

113299

In [46]:
from sklearn.model_selection import train_test_split 

X_train,X_test,y_train, y_test = train_test_split(X_sent_w, Y_label, test_size=0.2, random_state=42)
U_train,U_test,y_u_train, y_u_test = train_test_split(U_sent_w, Y_u_label, test_size=0.2, random_state=42)

In [47]:
ns_train = []
ns_test = []

ns_u_train = []
ns_u_test = []

for i in range(X_train.shape[0]):
    ns_train.append(X_train[i][-1])
for i in range(X_test.shape[0]):
    ns_test.append(X_test[i][-1])
    
for i in range(U_train.shape[0]):
    ns_u_train.append(U_train[i][-1])
for i in range(U_test.shape[0]):
    ns_u_test.append(U_test[i][-1])

In [48]:
X_n_train = np.delete(X_train,len(X_train[1])-1,axis=1)
X_n_test  = np.delete(X_test,len(X_test[1])-1,axis=1)

U_n_train = np.delete(U_train,len(U_train[1])-1,axis=1)
U_n_test  = np.delete(U_test,len(U_test[1])-1,axis=1)

In [49]:
# Now we define our model
from __future__ import print_function
from __future__ import division

import tensorflow as tf

def embedding_layer(ids_, V, embed_dim, init_scale=0.001):
    
    W_embed_ = tf.get_variable(name = 'W_embed', shape=[V,embed_dim],dtype=tf.float32
                               ,initializer=tf.random_uniform_initializer(minval= -init_scale
                                                                          ,maxval =init_scale),trainable=True)
    xs_ = tf.nn.embedding_lookup(W_embed_, ids_)
    return xs_

In [50]:
def fully_connected_layers(h0_, hidden_dims, activation=tf.tanh,
                           dropout_rate=0, is_training=False):
    h_ = h0_
    for i, hdim in enumerate(hidden_dims):
        
        h_ = tf.layers.dense(h_, hdim, activation=activation,    
                             kernel_initializer=tf.contrib.layers.xavier_initializer()
                             , bias_initializer=tf.zeros_initializer(), name = 'Hidden_%d'%i)
        
        if dropout_rate > 0:
            h_ = tf.nn.dropout(h_, keep_prob=dropout_rate)
            
        #if dropout_rate > 0:
            #h_ = tf.layers.dropout(inputs = h_, rate=dropout_rate, training = is_training)
        
    return h_

In [51]:
def softmax_output_layer(h_, labels_, num_classes):
    
    n = h_.get_shape().as_list()
    
    with tf.variable_scope("Logits"):
        W_out_ = tf.get_variable(name = 'W_out', shape=[n[1],num_classes],dtype=tf.float32
                               ,initializer=tf.random_normal_initializer(),trainable=True)
        b_out_ = tf.get_variable(name = 'b_out', shape=[num_classes],dtype=tf.float32
                               ,initializer=tf.random_normal_initializer(),trainable=True)
        logits_ = tf.matmul(h_,W_out_) + b_out_
        
        

    # If no labels provided, don't try to compute loss.
    if labels_ is None:
        return None, logits_

    with tf.name_scope("Softmax"):
        loss_ = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_,logits=logits_)) 
        
    
    return loss_, logits_

In [52]:
def BOW_encoder(ids_, ns_, V, embed_dim, hidden_dims, dropout_rate=0,is_training=None,
                **unused_kw):
    
    assert is_training is not None, "is_training must be explicitly set to True or False"
    
    with tf.variable_scope("Embedding_Layer"):
        xs_ = embedding_layer(ids_, V, embed_dim, init_scale=0.001) 
    mask_ = tf.expand_dims(tf.sequence_mask(ns_, xs_.shape[1],dtype=tf.float32), -1)
    xs_ =  mask_ * xs_
    x_ = tf.reduce_sum(xs_,axis = 1)
    h_ = fully_connected_layers(x_, hidden_dims, activation=tf.tanh, dropout_rate=dropout_rate,is_training=is_training)   
        
    return h_, xs_

In [53]:
def classifier_model_fn(features, labels, mode, params):
    # Seed the RNG for repeatability
    tf.set_random_seed(params.get('rseed', 10))

    # Check if this graph is going to be used for training.
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    if params['encoder_type'] == 'bow':
        with tf.variable_scope("Encoder"):
            h_, xs_ = BOW_encoder(features['ids'], features['ns'],
                                  is_training=is_training,
                                  **params)
    else:
        raise ValueError("Error: unsupported encoder type "
                         "'{:s}'".format(params['encoder_type']))

    # Construct softmax layer and loss functions
    with tf.variable_scope("Output_Layer"):
        ce_loss_, logits_ = softmax_output_layer(h_, labels, params['num_classes'])

    with tf.name_scope("Prediction"):
        pred_proba_ = tf.nn.softmax(logits_, name="pred_proba")
        pred_max_ = tf.argmax(logits_, 1, name="pred_max")
        predictions_dict = {"proba": pred_proba_, "max": pred_max_}

    if mode == tf.estimator.ModeKeys.PREDICT:
        # If predict mode, don't bother computing loss
        return tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=predictions_dict)

    # L2 regularization (weight decay) on parameters, from all layers
    with tf.variable_scope("Regularization"):
        l2_penalty_ = tf.nn.l2_loss(xs_)  # l2 loss on embeddings
        for var_ in tf.trainable_variables():
            if "Embedding_Layer" in var_.name:
                continue
            l2_penalty_ += tf.nn.l2_loss(var_)
        l2_penalty_ *= params['beta']  # scale by regularization strength
        tf.summary.scalar("l2_penalty", l2_penalty_)
        regularized_loss_ = ce_loss_ + l2_penalty_

    with tf.variable_scope("Training"):
        if params['optimizer'] == 'adagrad':
            optimizer_ = tf.train.AdagradOptimizer(params['lr'])
        else:
            optimizer_ = tf.train.GradientDescentOptimizer(params['lr'])
        train_op_ = optimizer_.minimize(regularized_loss_,
                                        global_step=tf.train.get_global_step())

    tf.summary.scalar("cross_entropy_loss", ce_loss_)
    eval_metrics = {"cross_entropy_loss": tf.metrics.mean(ce_loss_),
                    "accuracy": tf.metrics.accuracy(labels, pred_max_)}

    return tf.estimator.EstimatorSpec(mode=mode,
                                      predictions=predictions_dict,
                                      loss=regularized_loss_,
                                      train_op=train_op_,
                                      eval_metric_ops=eval_metrics)

In [54]:
# training the model

batch_size = 5
e = 175 # embedding dimension
h = [32,64,128] # hidden dimensions
l = 0.001 # learning rate
num_epoch = 20 #no. of epochs

# Specify model hyperparameters as used by model_fn
model_params = dict(V=V, embed_dim=e, hidden_dims=h, num_classes=3,encoder_type='bow',lr=l,optimizer='adagrad'
                    , beta=0.001)
model_fn = classifier_model_fn


with tf.Graph().as_default(), tf.Session() as sess:
    
    x_ph_  = tf.placeholder(tf.int32, shape=[None, X_n_train.shape[1]])  # [batch_size, max_len]
    ns_ph_ = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
    y_ph_  = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
    
    # Construct the graph using model_fn
    features = {"ids": x_ph_, "ns": ns_ph_}  # note that values are Tensors
    estimator_spec = model_fn(features, labels=y_ph_, mode=tf.estimator.ModeKeys.TRAIN,
                              params=model_params)
    loss_     = estimator_spec.loss
    train_op_ = estimator_spec.train_op
    
    
    sess.run(tf.global_variables_initializer())
    
    total_loss = 0
    loss_ema = np.log(2)  # track exponential-moving-average of loss
    ema_decay = np.exp(-1/10)  # decay parameter for moving average = np.exp(-1/history_length)
    
    
    y_conf = []
     
    for i in range(len(y_train)):
        y_conf.append(2 - np.absolute(Y_weak[i]-y_train[i]))
       
           
    
    y_conf = np.reshape(y_conf,newshape=(len(y_conf),))
    ns_train = np.reshape(ns_train,newshape=(len(ns_train),))
    
    for j in range(num_epoch):
        
        t0 = time.time()
        total_batches = 0
        total_examples = 0
        
        for (bx, bns, by) in utils.multi_batch_generator(batch_size, X_n_train, ns_train, y_conf):
            # feed NumPy arrays into the placeholder Tensors
            feed_dict = {x_ph_: bx, ns_ph_: bns, y_ph_: by}
            batch_loss, _ = sess.run([loss_, train_op_], feed_dict=feed_dict)
        
            # Compute some statistics
            total_batches += 1
            total_examples += len(bx)
            total_loss += batch_loss * len(bx)  # re-scale, since batch loss is mean
            # Compute moving average to smooth out noisy per-batch loss
            loss_ema = ema_decay * loss_ema + (1 - ema_decay) * batch_loss
        
            if (total_batches % 100 == 0):
                print("{:5,} examples, moving-average loss {:.2f}".format(total_examples, 
                                                                      loss_ema))    
        print("Completed no. {:d} epoch in {:s}".format(j+1,utils.pretty_timedelta(since=t0)))


  500 examples, moving-average loss 1.21
1,000 examples, moving-average loss 1.22
1,500 examples, moving-average loss 1.09
2,000 examples, moving-average loss 1.14
2,500 examples, moving-average loss 1.24
3,000 examples, moving-average loss 1.23
Completed no. 1 epoch in 0:00:05
  500 examples, moving-average loss 0.79
1,000 examples, moving-average loss 0.88
1,500 examples, moving-average loss 0.84
2,000 examples, moving-average loss 0.88
2,500 examples, moving-average loss 0.97
3,000 examples, moving-average loss 1.04
Completed no. 2 epoch in 0:00:02
  500 examples, moving-average loss 0.60
1,000 examples, moving-average loss 0.67
1,500 examples, moving-average loss 0.66
2,000 examples, moving-average loss 0.64
2,500 examples, moving-average loss 0.70
3,000 examples, moving-average loss 0.79
Completed no. 3 epoch in 0:00:02
  500 examples, moving-average loss 0.46
1,000 examples, moving-average loss 0.50
1,500 examples, moving-average loss 0.51
2,000 examples, moving-average loss 0.46

In [55]:
with tf.Graph().as_default(), tf.device('/gpu:0'),tf.Session() as sess:
    
    x_ph_  = tf.placeholder(tf.int32, shape=[None, X_n_test.shape[1]])  
    ns_ph_ = tf.placeholder(tf.int32, shape=[None])              
    y_ph_  = tf.placeholder(tf.int32, shape=[None])             
    
    # Construct the graph using model_fn
    features = {"ids": x_ph_, "ns": ns_ph_}  # note that values are Tensors
    estimator_spec = model_fn(features, labels=y_ph_, mode=tf.estimator.ModeKeys.PREDICT,
                              params=model_params)
    predict_ = estimator_spec.predictions
    
    sess.run(tf.global_variables_initializer())
    
    t = X_n_train.shape[0]
    
    true_conf_score = []
    for i in range(X_n_test.shape[0]):
        true_conf_score.append(2 - np.absolute(Y_weak[t+i]-y_test[i]))
    
    
    feed_dict = {x_ph_: X_n_test, ns_ph_: ns_test, y_ph_: true_conf_score}
    predictions = sess.run(predict_, feed_dict=feed_dict)

In [56]:
true_conf_score.count(0)

45

In [57]:
pred_conf_score = predictions['max']
len(true_conf_score)

866

In [58]:
conf_prob = predictions['proba']

t = X_n_train.shape[0]

for i in range(X_n_test.shape[0]):
    print(conf_prob[i]," test_label:",y_test[i]," weak_label:",Y_weak[t+i])

[ 0.07563666  0.03957781  0.88478547]  test_label: 2  weak_label: 2
[ 0.07553412  0.04512538  0.87934053]  test_label: 0  weak_label: 2
[ 0.07556286  0.04638939  0.8780477 ]  test_label: 2  weak_label: 1
[ 0.07301473  0.04259972  0.88438553]  test_label: 2  weak_label: 0
[ 0.07578142  0.04424974  0.87996882]  test_label: 0  weak_label: 1
[ 0.07572686  0.04819401  0.87607908]  test_label: 2  weak_label: 1
[ 0.07385767  0.0433814   0.88276094]  test_label: 0  weak_label: 1
[ 0.07525578  0.04670056  0.87804365]  test_label: 0  weak_label: 1
[ 0.07503267  0.04763155  0.87733585]  test_label: 0  weak_label: 1
[ 0.07923279  0.04768939  0.87307775]  test_label: 0  weak_label: 0
[ 0.07427606  0.04396876  0.88175517]  test_label: 1  weak_label: 1
[ 0.08033441  0.04677994  0.87288564]  test_label: 0  weak_label: 0
[ 0.07378038  0.04369602  0.88252366]  test_label: 0  weak_label: 0
[ 0.07650818  0.04507896  0.8784129 ]  test_label: 2  weak_label: 1
[ 0.07965148  0.04890124  0.87144727]  test_labe

[ 0.08128089  0.04423175  0.8744874 ]  test_label: 2  weak_label: 1
[ 0.07728848  0.04820643  0.8745051 ]  test_label: 0  weak_label: 2
[ 0.07692692  0.04099018  0.88208288]  test_label: 0  weak_label: 0
[ 0.07747971  0.04490609  0.8776142 ]  test_label: 0  weak_label: 1
[ 0.07465222  0.04539391  0.87995386]  test_label: 2  weak_label: 0
[ 0.07610068  0.04681165  0.87708759]  test_label: 0  weak_label: 1
[ 0.07548527  0.0462532   0.87826157]  test_label: 0  weak_label: 1
[ 0.0792285   0.04691809  0.87385339]  test_label: 1  weak_label: 1
[ 0.08244811  0.04329251  0.87425941]  test_label: 1  weak_label: 1
[ 0.07301024  0.04385043  0.88313931]  test_label: 0  weak_label: 1
[ 0.08317853  0.04921649  0.86760497]  test_label: 2  weak_label: 1
[ 0.07973189  0.04700594  0.87326217]  test_label: 0  weak_label: 1
[ 0.08100265  0.04659373  0.87240368]  test_label: 0  weak_label: 1
[ 0.0740388   0.04453143  0.88142979]  test_label: 2  weak_label: 1
[ 0.07134442  0.04303901  0.8856166 ]  test_labe

[ 0.07506081  0.0492946   0.87564462]  test_label: 1  weak_label: 1
[ 0.0750986   0.0461377   0.87876368]  test_label: 1  weak_label: 1
[ 0.07576087  0.04607601  0.8781631 ]  test_label: 0  weak_label: 1
[ 0.07734816  0.04551235  0.87713951]  test_label: 2  weak_label: 1
[ 0.07864197  0.05053021  0.87082785]  test_label: 0  weak_label: 1
[ 0.08006641  0.04636746  0.87356609]  test_label: 0  weak_label: 1
[ 0.07468905  0.04515138  0.88015956]  test_label: 0  weak_label: 1
[ 0.0807399   0.04849006  0.87076998]  test_label: 2  weak_label: 1
[ 0.07645342  0.0447925   0.87875402]  test_label: 0  weak_label: 1
[ 0.07454965  0.04597234  0.87947798]  test_label: 0  weak_label: 1
[ 0.07159203  0.04195868  0.88644928]  test_label: 2  weak_label: 1
[ 0.07789113  0.04536664  0.87674218]  test_label: 0  weak_label: 1
[ 0.07656067  0.04524121  0.87819809]  test_label: 2  weak_label: 1
[ 0.07752067  0.04588843  0.87659097]  test_label: 0  weak_label: 1
[ 0.07643469  0.04455657  0.87900871]  test_labe

[ 0.0773377   0.04552537  0.87713695]  test_label: 0  weak_label: 1
[ 0.07769004  0.04935799  0.87295192]  test_label: 1  weak_label: 1
[ 0.07939433  0.04555202  0.87505364]  test_label: 2  weak_label: 1
[ 0.07350338  0.0439159   0.88258076]  test_label: 0  weak_label: 1
[ 0.0771735   0.04898045  0.87384611]  test_label: 2  weak_label: 1
[ 0.07889624  0.04381067  0.87729305]  test_label: 2  weak_label: 1
[ 0.07063881  0.04430347  0.88505769]  test_label: 0  weak_label: 1
[ 0.08401738  0.04866079  0.86732179]  test_label: 0  weak_label: 1
[ 0.07520972  0.04344755  0.88134277]  test_label: 1  weak_label: 1
[ 0.07711944  0.04394905  0.87893146]  test_label: 2  weak_label: 1
[ 0.07867444  0.05051434  0.87081122]  test_label: 1  weak_label: 1
[ 0.07410834  0.04599859  0.879893  ]  test_label: 0  weak_label: 1
[ 0.0795842   0.04603009  0.87438571]  test_label: 0  weak_label: 2
[ 0.08211089  0.04164486  0.87624425]  test_label: 2  weak_label: 1
[ 0.07401855  0.04436091  0.88162053]  test_labe

In [59]:
num_correct = [pred_conf_score == true_conf_score]
sum(sum(num_correct))/len(pred_conf_score)

0.18591224018475749

In [60]:
model = tf.estimator.Estimator(model_fn=classifier_model_fn, 
                               params=model_params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_model_dir': '/tmp/tmpkjugavbd', '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_save_summary_steps': 100}


In [61]:
# Training params, just used in this cell for the input_fn-s
train_params = dict(batch_size=5, total_epochs=num_epoch, eval_every=1)
assert(train_params['total_epochs'] % train_params['eval_every'] == 0)

# Construct and train the model, saving checkpoints to the directory above.

ns_train = np.reshape(ns_train,newshape=(len(ns_train),))
y_conf = np.reshape(y_conf,newshape=(len(y_conf),))


train_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": X_n_train, "ns": ns_train}, y=y_conf,
                    batch_size=train_params['batch_size'], 
                    num_epochs=train_params['eval_every'], shuffle=True, seed=42
                 )

ns_test = np.reshape(ns_test,newshape=(len(ns_test),))
true_conf_score = np.reshape(true_conf_score,newshape=(len(true_conf_score),))

test_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": X_n_test, "ns": ns_test}, y=true_conf_score,
                    batch_size=20, num_epochs=1, shuffle=False
                )

true_conf_score = np.reshape(true_conf_score,newshape=(len(true_conf_score),))

In [62]:
with tf.device('/gpu:0'):
    for _ in range(train_params['total_epochs'] // train_params['eval_every']):
    # Train for a few epochs, then evaluate on test
        model.train(input_fn=train_input_fn)
        eval_metrics = model.evaluate(input_fn=test_input_fn, name="test")

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpkjugavbd/model.ckpt.
INFO:tensorflow:step = 1, loss = 1.62941
INFO:tensorflow:global_step/sec: 270.605
INFO:tensorflow:step = 101, loss = 1.02966 (0.371 sec)
INFO:tensorflow:global_step/sec: 289.073
INFO:tensorflow:step = 201, loss = 1.38245 (0.346 sec)
INFO:tensorflow:global_step/sec: 292.566
INFO:tensorflow:step = 301, loss = 1.02012 (0.342 sec)
INFO:tensorflow:global_step/sec: 286.692
INFO:tensorflow:step = 401, loss = 0.861883 (0.349 sec)
INFO:tensorflow:global_step/sec: 283.556
INFO:tensorflow:step = 501, loss = 0.857907 (0.352 sec)
INFO:tensorflow:global_step/sec: 295.533
INFO:tensorflow:step = 601, loss = 0.845381 (0.338 sec)
INFO:tensorflow:Saving checkpoints for 692 into /tmp/tmpkjugavbd/model.ckpt.
INFO:tensorflow:Loss for final step: 1.5416.
INFO:tensorflow:Starting evaluation at 2018-04-12-21:32:34
INFO:tensorflow:Restoring parameters from /tmp/tmpkjugavbd/model.ckpt-692
INFO:t

INFO:tensorflow:global_step/sec: 293.733
INFO:tensorflow:step = 4653, loss = 0.3061 (0.341 sec)
INFO:tensorflow:global_step/sec: 296.143
INFO:tensorflow:step = 4753, loss = 0.29863 (0.338 sec)
INFO:tensorflow:Saving checkpoints for 4844 into /tmp/tmpkjugavbd/model.ckpt.
INFO:tensorflow:Loss for final step: 0.308115.
INFO:tensorflow:Starting evaluation at 2018-04-12-21:33:00
INFO:tensorflow:Restoring parameters from /tmp/tmpkjugavbd/model.ckpt-4844
INFO:tensorflow:Finished evaluation at 2018-04-12-21:33:00
INFO:tensorflow:Saving dict for global step 4844: accuracy = 0.640878, cross_entropy_loss = 1.2532, global_step = 4844, loss = 1.53444
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpkjugavbd/model.ckpt-4844
INFO:tensorflow:Saving checkpoints for 4845 into /tmp/tmpkjugavbd/model.ckpt.
INFO:tensorflow:step = 4845, loss = 0.302555
INFO:tensorflow:global_step/sec: 288.691
INFO:tensorflow:step = 4945, loss = 0.289585 (0.348 sec)
INFO:tensorflo

INFO:tensorflow:Saving dict for global step 8996: accuracy = 0.635104, cross_entropy_loss = 1.44721, global_step = 8996, loss = 1.72429
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpkjugavbd/model.ckpt-8996
INFO:tensorflow:Saving checkpoints for 8997 into /tmp/tmpkjugavbd/model.ckpt.
INFO:tensorflow:step = 8997, loss = 0.284645
INFO:tensorflow:global_step/sec: 286.299
INFO:tensorflow:step = 9097, loss = 0.279055 (0.351 sec)
INFO:tensorflow:global_step/sec: 296.956
INFO:tensorflow:step = 9197, loss = 0.29215 (0.337 sec)
INFO:tensorflow:global_step/sec: 302.575
INFO:tensorflow:step = 9297, loss = 0.289483 (0.330 sec)
INFO:tensorflow:global_step/sec: 305.642
INFO:tensorflow:step = 9397, loss = 0.279636 (0.327 sec)
INFO:tensorflow:global_step/sec: 307.739
INFO:tensorflow:step = 9497, loss = 0.283694 (0.325 sec)
INFO:tensorflow:global_step/sec: 313.278
INFO:tensorflow:step = 9597, loss = 0.281607 (0.319 sec)
INFO:tensorflow:Saving checkpoints 

INFO:tensorflow:global_step/sec: 288.401
INFO:tensorflow:step = 13349, loss = 0.282579 (0.347 sec)
INFO:tensorflow:global_step/sec: 289.616
INFO:tensorflow:step = 13449, loss = 0.280735 (0.345 sec)
INFO:tensorflow:global_step/sec: 287.688
INFO:tensorflow:step = 13549, loss = 0.274273 (0.348 sec)
INFO:tensorflow:global_step/sec: 284.641
INFO:tensorflow:step = 13649, loss = 0.27703 (0.352 sec)
INFO:tensorflow:global_step/sec: 289.482
INFO:tensorflow:step = 13749, loss = 0.275591 (0.345 sec)
INFO:tensorflow:Saving checkpoints for 13840 into /tmp/tmpkjugavbd/model.ckpt.
INFO:tensorflow:Loss for final step: 0.277007.
INFO:tensorflow:Starting evaluation at 2018-04-12-21:33:54
INFO:tensorflow:Restoring parameters from /tmp/tmpkjugavbd/model.ckpt-13840
INFO:tensorflow:Finished evaluation at 2018-04-12-21:33:55
INFO:tensorflow:Saving dict for global step 13840: accuracy = 0.642032, cross_entropy_loss = 1.53416, global_step = 13840, loss = 1.80626


In [63]:
eval_metrics = model.evaluate(input_fn=test_input_fn, name="test")  # replace with result of model.evaluate(...)


print("Accuracy on test set: {:.02%}".format(eval_metrics['accuracy']))
eval_metrics

INFO:tensorflow:Starting evaluation at 2018-04-12-21:34:08
INFO:tensorflow:Restoring parameters from /tmp/tmpkjugavbd/model.ckpt-13840
INFO:tensorflow:Finished evaluation at 2018-04-12-21:34:08
INFO:tensorflow:Saving dict for global step 13840: accuracy = 0.642032, cross_entropy_loss = 1.53416, global_step = 13840, loss = 1.80626
Accuracy on test set: 64.20%


{'accuracy': 0.64203233,
 'cross_entropy_loss': 1.5341592,
 'global_step': 13840,
 'loss': 1.8062626}

In [64]:
def softmax_output_layer_conf(h_, labels_, num_classes, conf):
    
    n = h_.get_shape().as_list()
    
    with tf.variable_scope("Logits"):
        W_out_ = tf.get_variable(name = 'W_out', shape=[n[1],num_classes],dtype=tf.float32
                               ,initializer=tf.random_normal_initializer(),trainable=True)
        b_out_ = tf.get_variable(name = 'b_out', shape=[num_classes],dtype=tf.float32
                               ,initializer=tf.random_normal_initializer(),trainable=True)
        logits_ = tf.matmul(h_,W_out_) + b_out_
        
        

    # If no labels provided, don't try to compute loss.
    if labels_ is None:
        return None, logits_

    with tf.name_scope("Softmax"):
        #c = tf.constant(conf,dtype = tf.float32)
        loss_ = tf.reduce_mean(tf.cast(conf,dtype = tf.float32) * tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_,logits=logits_)) 
        
    
    return loss_, logits_

In [65]:
def classifier_model_fn_conf(features, labels, mode, params):
    # Seed the RNG for repeatability
    tf.set_random_seed(params.get('rseed', 10))

    # Check if this graph is going to be used for training.
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    if params['encoder_type'] == 'bow':
        with tf.variable_scope("Encoder"):
            h_, xs_ = BOW_encoder(features['ids'], features['ns'],
                                  is_training=is_training,
                                  **params)
    else:
        raise ValueError("Error: unsupported encoder type "
                         "'{:s}'".format(params['encoder_type']))

    # Construct softmax layer and loss functions
    with tf.variable_scope("Output_Layer"):
        ce_loss_, logits_ = softmax_output_layer_conf(h_, labels, params['num_classes'],conf=features['conf'])

    with tf.name_scope("Prediction"):
        pred_proba_ = tf.nn.softmax(logits_, name="pred_proba")
        pred_max_ = tf.argmax(logits_, 1, name="pred_max")
        predictions_dict = {"proba": pred_proba_, "max": pred_max_}

    if mode == tf.estimator.ModeKeys.PREDICT:
        # If predict mode, don't bother computing loss.
        return tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=predictions_dict)

    # L2 regularization (weight decay) on parameters, from all layers
    with tf.variable_scope("loss"):
        l2_penalty_ = tf.nn.l2_loss(xs_)  # l2 loss on embeddings
        for var_ in tf.trainable_variables():
            if "Embedding_Layer" in var_.name:
                continue
            l2_penalty_ += tf.nn.l2_loss(var_)
        l2_penalty_ *= params['beta']  # scale by regularization strength
        tf.summary.scalar("l2_penalty", l2_penalty_)
        regularized_loss_ = ce_loss_ + l2_penalty_
        

    with tf.variable_scope("Training"):
        if params['optimizer'] == 'adagrad':
            optimizer_ = tf.train.AdagradOptimizer(params['lr'])
        else:
            optimizer_ = tf.train.GradientDescentOptimizer(params['lr'])
        train_op_ = optimizer_.minimize(regularized_loss_,
                                        global_step=tf.train.get_global_step())

    tf.summary.scalar("cross_entropy_loss", ce_loss_)
    eval_metrics = {"cross_entropy_loss": tf.metrics.mean(ce_loss_),
                    "accuracy": tf.metrics.accuracy(labels, pred_max_)}

    return tf.estimator.EstimatorSpec(mode=mode,
                                      predictions=predictions_dict,
                                      loss=regularized_loss_,
                                      train_op=train_op_,
                                      eval_metric_ops=eval_metrics)

In [66]:
true_conf_score = []
for i in range(U_n_train.shape[0]):
    true_conf_score.append(2)
    
ns_u_train = np.reshape(ns_u_train,newshape=(len(ns_u_train),))

train_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": U_n_train, "ns": ns_u_train},
                    batch_size=1000, num_epochs=1, shuffle=False
                )

predictions_train = model.predict(input_fn=train_input_fn)  

ns_u_test = np.reshape(ns_u_test,newshape=(len(ns_u_test),))

test_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": U_n_test, "ns": ns_u_test},
                    batch_size=1000, num_epochs=1, shuffle=False
                )

predictions_test = model.predict(input_fn=test_input_fn)

In [67]:
predictions_train =list(predictions_train)
predictions_test =list(predictions_test)

INFO:tensorflow:Restoring parameters from /tmp/tmpkjugavbd/model.ckpt-13840
INFO:tensorflow:Restoring parameters from /tmp/tmpkjugavbd/model.ckpt-13840


In [68]:
conf_score_u_train = []

for i in range(len(predictions_train)):
    conf_score_u_train.append(predictions_train[i]['max'])
    
conf_score_u_test = []

for i in range(len(predictions_test)):
    conf_score_u_test.append(predictions_test[i]['max'])

In [69]:
model_params = dict(V=V, embed_dim=e, hidden_dims=h, num_classes=3,encoder_type='bow',lr=l,optimizer='adagrad'
                    , beta=0.001)


model1 = tf.estimator.Estimator(model_fn=classifier_model_fn_conf, 
                               params=model_params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_model_dir': '/tmp/tmpqkbx6lwt', '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_save_summary_steps': 100}


In [70]:
# Training the U dataset
train_params = dict(batch_size=250, total_epochs=num_epoch, eval_every=1)
assert(train_params['total_epochs'] % train_params['eval_every'] == 0)

# Construct and train the model, saving checkpoints to the directory above.

ns_u_train = np.reshape(ns_u_train,newshape=(len(ns_u_train),))
y_u_train = np.reshape(y_u_train,newshape=(len(y_u_train),))
conf_score_u_train = np.reshape(conf_score_u_train,newshape=(U_n_train.shape[0],))

train_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": U_n_train, "ns": ns_u_train,"conf": conf_score_u_train}, y=y_u_train,
                    batch_size=train_params['batch_size'], 
                    num_epochs=train_params['eval_every'], shuffle=True, seed=42
                 )

ns_u_test = np.reshape(ns_u_test,newshape=(len(ns_u_test),))
y_u_test = np.reshape(y_u_test,newshape=(len(y_u_test),))
conf_score_u_test = np.reshape(conf_score_u_test,newshape=(U_n_test.shape[0],))

test_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": U_n_test, "ns": ns_u_test,"conf": conf_score_u_test}, y=y_u_test,
                    batch_size=1000, num_epochs=1, shuffle=False
                )

In [71]:
with tf.device('/device:GPU:0'):
    for _ in range(train_params['total_epochs'] // train_params['eval_every']):
    # Train for a few epochs, then evaluate on test
        model1.train(input_fn=train_input_fn)
        eval_metrics = model1.evaluate(input_fn=test_input_fn, name="test")

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpqkbx6lwt/model.ckpt.
INFO:tensorflow:step = 1, loss = 1.71607
INFO:tensorflow:global_step/sec: 11.6391
INFO:tensorflow:step = 101, loss = 0.621098 (8.593 sec)
INFO:tensorflow:global_step/sec: 11.6516
INFO:tensorflow:step = 201, loss = 0.511374 (8.582 sec)
INFO:tensorflow:global_step/sec: 11.6336
INFO:tensorflow:step = 301, loss = 0.52165 (8.596 sec)
INFO:tensorflow:Saving checkpoints for 363 into /tmp/tmpqkbx6lwt/model.ckpt.
INFO:tensorflow:Loss for final step: 0.481827.
INFO:tensorflow:Starting evaluation at 2018-04-12-21:34:55
INFO:tensorflow:Restoring parameters from /tmp/tmpqkbx6lwt/model.ckpt-363
INFO:tensorflow:Finished evaluation at 2018-04-12-21:34:58
INFO:tensorflow:Saving dict for global step 363: accuracy = 0.952251, cross_entropy_loss = 0.202962, global_step = 363, loss = 0.492254
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpqkbx

INFO:tensorflow:step = 3105, loss = 0.314914 (8.081 sec)
INFO:tensorflow:global_step/sec: 12.2234
INFO:tensorflow:step = 3205, loss = 0.318273 (8.181 sec)
INFO:tensorflow:Saving checkpoints for 3267 into /tmp/tmpqkbx6lwt/model.ckpt.
INFO:tensorflow:Loss for final step: 0.317086.
INFO:tensorflow:Starting evaluation at 2018-04-12-21:39:42
INFO:tensorflow:Restoring parameters from /tmp/tmpqkbx6lwt/model.ckpt-3267
INFO:tensorflow:Finished evaluation at 2018-04-12-21:39:46
INFO:tensorflow:Saving dict for global step 3267: accuracy = 0.981112, cross_entropy_loss = 0.0745269, global_step = 3267, loss = 0.361617
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpqkbx6lwt/model.ckpt-3267
INFO:tensorflow:Saving checkpoints for 3268 into /tmp/tmpqkbx6lwt/model.ckpt.
INFO:tensorflow:step = 3268, loss = 0.418702
INFO:tensorflow:global_step/sec: 11.4856
INFO:tensorflow:step = 3368, loss = 0.316922 (8.708 sec)
INFO:tensorflow:global_step/sec: 11.2539
INFO:te

INFO:tensorflow:Starting evaluation at 2018-04-12-21:44:33
INFO:tensorflow:Restoring parameters from /tmp/tmpqkbx6lwt/model.ckpt-6171
INFO:tensorflow:Finished evaluation at 2018-04-12-21:44:37
INFO:tensorflow:Saving dict for global step 6171: accuracy = 0.983672, cross_entropy_loss = 0.0665147, global_step = 6171, loss = 0.349638
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpqkbx6lwt/model.ckpt-6171
INFO:tensorflow:Saving checkpoints for 6172 into /tmp/tmpqkbx6lwt/model.ckpt.
INFO:tensorflow:step = 6172, loss = 0.356843
INFO:tensorflow:global_step/sec: 12.2306
INFO:tensorflow:step = 6272, loss = 0.291685 (8.178 sec)
INFO:tensorflow:global_step/sec: 12.2119
INFO:tensorflow:step = 6372, loss = 0.287811 (8.189 sec)
INFO:tensorflow:global_step/sec: 12.4524
INFO:tensorflow:step = 6472, loss = 0.289463 (8.030 sec)
INFO:tensorflow:Saving checkpoints for 6534 into /tmp/tmpqkbx6lwt/model.ckpt.
INFO:tensorflow:Loss for final step: 0.289304.
INFO:te

In [72]:
conf_score_u = list(conf_score_u_train) + list(conf_score_u_test)
conf_score_u = np.reshape(conf_score_u,newshape = (len(conf_score_u),))

predict_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": U_sent, "ns":u_ns , "conf": conf_score_u},
                    batch_size=20, num_epochs=1, shuffle=False
                )

predictions = model1.predict(input_fn=predict_input_fn)

In [73]:
predictions = list(predictions)
pred_label_U = []

for i in range(len(predictions)):
    pred_label_U.append(predictions[i]['max'])

INFO:tensorflow:Restoring parameters from /tmp/tmpqkbx6lwt/model.ckpt-7260


In [74]:
eval_metrics = model1.evaluate(input_fn=test_input_fn, name="test")  # replace with result of model.evaluate(...)


print("Accuracy on test set: {:.02%}".format(eval_metrics['accuracy']))
eval_metrics

INFO:tensorflow:Starting evaluation at 2018-04-12-21:53:30
INFO:tensorflow:Restoring parameters from /tmp/tmpqkbx6lwt/model.ckpt-7260
INFO:tensorflow:Finished evaluation at 2018-04-12-21:53:34
INFO:tensorflow:Saving dict for global step 7260: accuracy = 0.984157, cross_entropy_loss = 0.0665251, global_step = 7260, loss = 0.348155
Accuracy on test set: 98.42%


{'accuracy': 0.98415709,
 'cross_entropy_loss': 0.066525094,
 'global_step': 7260,
 'loss': 0.34815535}

In [75]:
pred_label_U = pd.DataFrame(data = pred_label_U, columns = ['pred_labels'])
pred_label_U.to_csv("predictions_12April.csv")

In [76]:
pred_label_U = pd.read_csv("predictions_12April.csv")

In [77]:
pred_label_U.count()

Unnamed: 0     113299
pred_labels    113299
dtype: int64

In [78]:
pred_label_U[pred_label_U == 2].count()

Unnamed: 0        1
pred_labels    1704
dtype: int64

In [79]:
pred_label_U[pred_label_U == 1].count()

Unnamed: 0         1
pred_labels    92233
dtype: int64

In [80]:
pred_label_U[pred_label_U == 0].count()

Unnamed: 0         1
pred_labels    19362
dtype: int64

In [81]:
pred_label_U_0 = []

for i in range(len(pred_label_U)):
    if pred_label_U.loc[i]['pred_labels'] == 0:
        pred_label_U_0.append(1)
    else:
        pred_label_U_0.append(0)

In [82]:
pred_label_U_1 = []

for i in range(len(pred_label_U)):
    if pred_label_U.loc[i]['pred_labels'] == 1:
        pred_label_U_1.append(1)
    else:
        pred_label_U_1.append(0)

In [83]:
pred_label_U_2 = []

for i in range(len(pred_label_U)):
    if pred_label_U.loc[i]['pred_labels'] == 2:
        pred_label_U_2.append(1)
    else:
        pred_label_U_2.append(0)

In [84]:
pred_label_U['label_0'] = pred_label_U_0

In [85]:
pred_label_U['label_1'] = pred_label_U_1

In [86]:
pred_label_U['label_2'] = pred_label_U_2

In [87]:
feature_size = U_sent.shape[1]


In [88]:
# Placeholders

input_x = tf.placeholder(tf.float32,shape=[None,feature_size])

In [89]:
num_classes = 3

In [90]:
input_y = tf.placeholder(tf.float32,shape=[None,num_classes])

In [91]:
hold_prob = tf.placeholder(tf.float32, name="hold_prob")

In [92]:
# Now we define our model
from __future__ import print_function
from __future__ import division

import tensorflow as tf

def embedding_layer(ids_, V, embed_dim, init_scale=0.001):
    
    W_embed_ = tf.get_variable(name = 'W_embed', shape=[V,embed_dim],dtype=tf.float32
                               ,initializer=tf.random_uniform_initializer(minval= -init_scale
                                                                          ,maxval =init_scale),trainable=True)
    xs_ = tf.nn.embedding_lookup(W_embed_, ids_)
    return xs_

In [93]:
def fully_connected_layers(h0_, hidden_dims, activation=tf.tanh,
                           dropout_rate=0, is_training=False):
    h_ = h0_
    for i, hdim in enumerate(hidden_dims):
        
        h_ = tf.layers.dense(h_, hdim, activation=activation,    
                             kernel_initializer=tf.contrib.layers.xavier_initializer()
                             , bias_initializer=tf.zeros_initializer(), name = 'Hidden_%d'%i)
        
        if dropout_rate > 0:
            h_ = tf.nn.dropout(h_, keep_prob=dropout_rate)
            
        #if dropout_rate > 0:
            #h_ = tf.layers.dropout(inputs = h_, rate=dropout_rate, training = is_training)
        
    return h_

In [94]:
def softmax_output_layer(h_, labels_, num_classes):
    
    n = h_.get_shape().as_list()
    
    with tf.variable_scope("Logits"):
        W_out_ = tf.get_variable(name = 'W_out', shape=[n[1],num_classes],dtype=tf.float32
                               ,initializer=tf.random_normal_initializer(),trainable=True)
        b_out_ = tf.get_variable(name = 'b_out', shape=[num_classes],dtype=tf.float32
                               ,initializer=tf.random_normal_initializer(),trainable=True)
        logits_ = tf.matmul(h_,W_out_) + b_out_
        
        

    # If no labels provided, don't try to compute loss.
    if labels_ is None:
        return None, logits_

    with tf.name_scope("Softmax"):
        loss_ = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_,logits=logits_)) 
        
    
    return loss_, logits_

In [95]:
class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(self, sequence_length, num_classes, vocab_size,
        embedding_size, filter_heights, filter_widths,num_filters,channels_in,channels_out, init_scale,l2_reg_lambda):

        assert(len(filter_widths)==num_filters)
        assert(len(filter_heights)==num_filters)
        assert(len(channels_in)==num_filters)
        assert(len(channels_out)==num_filters)
        assert(channels_in[0]==1)
        
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        
        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)
        
        # Embedding layer
        with tf.name_scope("embedding_CNN"):
            W_embedding = tf.get_variable(name = 'W_embedding', shape=[vocab_size,embedding_size],dtype=tf.float32
                               ,initializer=tf.random_uniform_initializer(minval= -init_scale
                                                                          ,maxval =init_scale),trainable=True)
            self.xs = tf.nn.embedding_lookup(W_embedding, self.input_x)
            self.xs_expanded = tf.reshape(self.xs,[-1,sequence_length,embedding_size,1])
            
        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        i = 1
        x = self.xs_expanded
        for filter_h, filter_w, c_in, c_out in zip(filter_heights,filter_widths,channels_in,channels_out):
            with tf.name_scope("conv-maxpool-%s" % i):
                # Convolution Layer
                filter_shape = [filter_h, filter_w, c_in , c_out]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[c_out]), name="b")
                conv = tf.nn.conv2d(x,W,strides=[1,1,1,1],padding="SAME",name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(h,ksize=[1,embedding_size-filter_w+1, 1, 1],strides=[1, 1, 1, 1],padding="SAME",name="pool")
                x = pooled
                i = i + 1
                pooled_outputs.append(pooled)
        
        length = len(pooled_outputs)
        # Combine all the pooled features
        num_filters_total = c_out*sequence_length*embedding_size
        self.h_pool_flat = tf.reshape(pooled_outputs[length-1], [-1, num_filters_total])
        
        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)
            
        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")
            
        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy") 


In [98]:
# final padded sentences
X_sent_test, ns = utils.pad_np_array(X_ids, max_len=max_len2, pad_id=0)

In [99]:
Y_label = pd.DataFrame(data = Y_label, columns = ['labels'])
label_0 = []

for i in range(len(Y_label)):
    if Y_label.loc[i]['labels'] == 0:
        label_0.append(1)
    else:
        label_0.append(0)

In [100]:
label_1 = []

for i in range(len(Y_label)):
    if Y_label.loc[i]['labels'] == 1:
        label_1.append(1)
    else:
        label_1.append(0)

In [101]:
label_2 = []

for i in range(len(Y_label)):
    if Y_label.loc[i]['labels'] == 2:
        label_2.append(1)
    else:
        label_2.append(0)

In [102]:
Y_label['label_0'] = label_0
Y_label['label_1'] = label_1
Y_label['label_2'] = label_2

In [106]:
Y_label.iloc[0:600][['label_0','label_1','label_2']]         

,label_0,label_1,label_2
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
5,1,0,0
6,1,0,0
7,1,0,0
8,1,0,0
9,1,0,0


In [111]:
batch_size = 100
num_epoch = 1
num_checkpoints = 5

with tf.Graph().as_default(), tf.Session() as sess:

    with sess.as_default():
        cnn = TextCNN(sequence_length=feature_size, num_classes=3, vocab_size=V,
        embedding_size=100, filter_heights=[5,5,5], filter_widths=[5,5,5],num_filters=3,channels_in=[1,10,12]
              ,channels_out=[10,12,24]
              ,init_scale=0.08, l2_reg_lambda=0.1)

        
        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)
        
        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)
        
        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)
        
        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 0.5
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)
            
        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
            
        from sklearn.model_selection import train_test_split 

        
        U_train,U_dev,y_train, y_dev = train_test_split(U_sent, pred_label_U[['label_0','label_1','label_2']], 
                                                        test_size=0.005, random_state=42)
        
        for j in range(num_epoch):
        
            t0 = time.time()
            total_batches = 0
            total_examples = 0
        
            for (bx, by) in utils.multi_batch_generator(batch_size, U_train, y_train):
                train_step(bx,by)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % 50 == 0:
                    print("\nEvaluation:")
                    dev_step(U_dev, y_dev, writer=dev_summary_writer)
                    print("")
                if current_step % 50 == 0:
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(path))
                    
        print("\nTest set accuracy:")
        dev_step(X_sent_test[0:600], Y_label.iloc[0:600][['label_0','label_1','label_2']], writer=dev_summary_writer)
        print("")

Writing to /home/ramya_girish/political_bias/political_bias/runs/1523578017

2018-04-13T00:07:01.329226: step 1, loss 1.50804, acc 0.29
2018-04-13T00:07:03.043461: step 2, loss 83.3398, acc 0.79
2018-04-13T00:07:04.741172: step 3, loss 98.4132, acc 0.75
2018-04-13T00:07:06.427617: step 4, loss 40.413, acc 0.86
2018-04-13T00:07:08.115939: step 5, loss 33.7409, acc 0.81
2018-04-13T00:07:09.785575: step 6, loss 24.1075, acc 0.78
2018-04-13T00:07:11.454745: step 7, loss 8.88641, acc 0.82
2018-04-13T00:07:13.117280: step 8, loss 24.996, acc 0.17
2018-04-13T00:07:14.784913: step 9, loss 21.0143, acc 0.22
2018-04-13T00:07:16.442222: step 10, loss 6.84779, acc 0.16
2018-04-13T00:07:18.093108: step 11, loss 3.32468, acc 0.85
2018-04-13T00:07:19.735124: step 12, loss 6.08347, acc 0.76
2018-04-13T00:07:21.383001: step 13, loss 5.67003, acc 0.79
2018-04-13T00:07:23.012329: step 14, loss 3.4453, acc 0.87
2018-04-13T00:07:24.631812: step 15, loss 4.89934, acc 0.74
2018-04-13T00:07:26.252500: step 16

2018-04-13T00:10:41.629501: step 130, loss 0.632039, acc 0.87
2018-04-13T00:10:43.226497: step 131, loss 0.631576, acc 0.85
2018-04-13T00:10:44.825001: step 132, loss 0.662599, acc 0.83
2018-04-13T00:10:46.427511: step 133, loss 0.622234, acc 0.85
2018-04-13T00:10:48.031641: step 134, loss 0.767864, acc 0.81
2018-04-13T00:10:49.638317: step 135, loss 0.612674, acc 0.85
2018-04-13T00:10:51.235963: step 136, loss 0.806828, acc 0.76
2018-04-13T00:10:52.838960: step 137, loss 0.665158, acc 0.82
2018-04-13T00:10:54.435650: step 138, loss 0.680604, acc 0.82
2018-04-13T00:10:56.032030: step 139, loss 0.663089, acc 0.83
2018-04-13T00:10:57.641961: step 140, loss 0.792895, acc 0.76
2018-04-13T00:10:59.244556: step 141, loss 0.694758, acc 0.81
2018-04-13T00:11:00.845533: step 142, loss 0.619723, acc 0.86
2018-04-13T00:11:02.446489: step 143, loss 0.625378, acc 0.86
2018-04-13T00:11:04.045816: step 144, loss 0.660026, acc 0.83
2018-04-13T00:11:05.655631: step 145, loss 0.595868, acc 0.85
2018-04-

2018-04-13T00:14:16.885317: step 254, loss 0.460036, acc 0.8
2018-04-13T00:14:18.486482: step 255, loss 0.49623, acc 0.79
2018-04-13T00:14:20.089166: step 256, loss 0.413141, acc 0.83
2018-04-13T00:14:21.683267: step 257, loss 0.320049, acc 0.85
2018-04-13T00:14:23.285045: step 258, loss 0.644897, acc 0.8
2018-04-13T00:14:24.883013: step 259, loss 0.38191, acc 0.94
2018-04-13T00:14:26.485775: step 260, loss 0.519313, acc 0.88
2018-04-13T00:14:28.082535: step 261, loss 0.44033, acc 0.91
2018-04-13T00:14:29.677802: step 262, loss 0.472724, acc 0.85
2018-04-13T00:14:31.276330: step 263, loss 0.392568, acc 0.87
2018-04-13T00:14:32.879531: step 264, loss 0.39722, acc 0.85
2018-04-13T00:14:34.482127: step 265, loss 0.442471, acc 0.84
2018-04-13T00:14:36.083143: step 266, loss 0.485596, acc 0.85
2018-04-13T00:14:37.688607: step 267, loss 0.380023, acc 0.86
2018-04-13T00:14:39.289338: step 268, loss 0.503239, acc 0.85
2018-04-13T00:14:40.889563: step 269, loss 0.399654, acc 0.89
2018-04-13T00:

2018-04-13T00:17:51.801490: step 381, loss 0.323364, acc 0.93
2018-04-13T00:17:53.408777: step 382, loss 0.195271, acc 0.97
2018-04-13T00:17:55.015744: step 383, loss 0.120112, acc 0.98
2018-04-13T00:17:56.620571: step 384, loss 0.232283, acc 0.95
2018-04-13T00:17:58.223746: step 385, loss 0.201995, acc 0.96
2018-04-13T00:17:59.832415: step 386, loss 0.0933379, acc 0.99
2018-04-13T00:18:01.430577: step 387, loss 0.277124, acc 0.93
2018-04-13T00:18:03.034560: step 388, loss 0.151955, acc 0.97
2018-04-13T00:18:04.643852: step 389, loss 0.226367, acc 0.95
2018-04-13T00:18:06.254788: step 390, loss 0.192059, acc 0.97
2018-04-13T00:18:07.861914: step 391, loss 0.108359, acc 0.97
2018-04-13T00:18:09.472471: step 392, loss 0.235684, acc 0.95
2018-04-13T00:18:11.078333: step 393, loss 0.271035, acc 0.94
2018-04-13T00:18:12.692949: step 394, loss 0.15365, acc 0.98
2018-04-13T00:18:14.292065: step 395, loss 0.234125, acc 0.95
2018-04-13T00:18:15.899488: step 396, loss 0.2495, acc 0.95
2018-04-13

2018-04-13T00:21:26.284532: step 504, loss 0.123894, acc 0.98
2018-04-13T00:21:27.895295: step 505, loss 0.15587, acc 0.98
2018-04-13T00:21:29.502985: step 506, loss 0.127394, acc 0.97
2018-04-13T00:21:31.108085: step 507, loss 0.139911, acc 0.96
2018-04-13T00:21:32.714056: step 508, loss 0.144783, acc 0.98
2018-04-13T00:21:34.316849: step 509, loss 0.107785, acc 0.99
2018-04-13T00:21:35.926287: step 510, loss 0.12533, acc 0.98
2018-04-13T00:21:37.547350: step 511, loss 0.125451, acc 0.98
2018-04-13T00:21:39.161353: step 512, loss 0.0902624, acc 0.98
2018-04-13T00:21:40.767879: step 513, loss 0.2449, acc 0.94
2018-04-13T00:21:42.385610: step 514, loss 0.149169, acc 0.98
2018-04-13T00:21:44.006678: step 515, loss 0.106997, acc 0.98
2018-04-13T00:21:45.610836: step 516, loss 0.219084, acc 0.95
2018-04-13T00:21:47.222246: step 517, loss 0.0590859, acc 1
2018-04-13T00:21:48.825277: step 518, loss 0.195921, acc 0.94
2018-04-13T00:21:50.434242: step 519, loss 0.161369, acc 0.96
2018-04-13T00

2018-04-13T00:25:00.345802: step 630, loss 0.102727, acc 0.99
2018-04-13T00:25:01.955538: step 631, loss 0.0875015, acc 0.99
2018-04-13T00:25:03.564065: step 632, loss 0.0559749, acc 0.99
2018-04-13T00:25:05.167712: step 633, loss 0.14655, acc 0.96
2018-04-13T00:25:06.775014: step 634, loss 0.173235, acc 0.96
2018-04-13T00:25:08.376722: step 635, loss 0.113918, acc 0.97
2018-04-13T00:25:09.989864: step 636, loss 0.0604975, acc 1
2018-04-13T00:25:11.605763: step 637, loss 0.121602, acc 0.97
2018-04-13T00:25:13.216408: step 638, loss 0.0897456, acc 0.98
2018-04-13T00:25:14.835716: step 639, loss 0.114762, acc 0.98
2018-04-13T00:25:16.443036: step 640, loss 0.0521811, acc 1
2018-04-13T00:25:18.054780: step 641, loss 0.125519, acc 0.97
2018-04-13T00:25:19.669037: step 642, loss 0.18074, acc 0.96
2018-04-13T00:25:21.276144: step 643, loss 0.171845, acc 0.96
2018-04-13T00:25:22.889385: step 644, loss 0.210064, acc 0.92
2018-04-13T00:25:24.507231: step 645, loss 0.112387, acc 0.97
2018-04-13T

2018-04-13T00:28:35.214767: step 753, loss 0.086914, acc 0.98
2018-04-13T00:28:36.819337: step 754, loss 0.144911, acc 0.97
2018-04-13T00:28:38.427739: step 755, loss 0.126201, acc 0.97
2018-04-13T00:28:40.039488: step 756, loss 0.0799959, acc 0.99
2018-04-13T00:28:41.649911: step 757, loss 0.087874, acc 0.98
2018-04-13T00:28:43.259362: step 758, loss 0.0599924, acc 1
2018-04-13T00:28:44.870182: step 759, loss 0.0590788, acc 0.99
2018-04-13T00:28:46.479255: step 760, loss 0.0548369, acc 0.99
2018-04-13T00:28:48.090464: step 761, loss 0.0452974, acc 1
2018-04-13T00:28:49.701401: step 762, loss 0.082007, acc 0.98
2018-04-13T00:28:51.314075: step 763, loss 0.0990955, acc 0.98
2018-04-13T00:28:52.933541: step 764, loss 0.063454, acc 0.99
2018-04-13T00:28:54.540868: step 765, loss 0.0844904, acc 0.98
2018-04-13T00:28:56.155728: step 766, loss 0.11125, acc 0.97
2018-04-13T00:28:57.769016: step 767, loss 0.0718185, acc 0.99
2018-04-13T00:28:59.373947: step 768, loss 0.133373, acc 0.96
2018-04

2018-04-13T00:32:10.993279: step 880, loss 0.067679, acc 0.97
2018-04-13T00:32:12.603885: step 881, loss 0.0603829, acc 0.98
2018-04-13T00:32:14.211776: step 882, loss 0.0510811, acc 0.99
2018-04-13T00:32:15.827943: step 883, loss 0.136312, acc 0.97
2018-04-13T00:32:17.434979: step 884, loss 0.21286, acc 0.97
2018-04-13T00:32:19.047409: step 885, loss 0.0864216, acc 0.97
2018-04-13T00:32:20.656776: step 886, loss 0.0432801, acc 1
2018-04-13T00:32:22.269921: step 887, loss 0.0718389, acc 0.97
2018-04-13T00:32:23.887660: step 888, loss 0.146995, acc 0.96
2018-04-13T00:32:25.501959: step 889, loss 0.123689, acc 0.97
2018-04-13T00:32:27.107552: step 890, loss 0.0929575, acc 0.99
2018-04-13T00:32:28.719715: step 891, loss 0.0387415, acc 1
2018-04-13T00:32:30.325484: step 892, loss 0.0485311, acc 0.99
2018-04-13T00:32:31.937688: step 893, loss 0.109648, acc 0.97
2018-04-13T00:32:33.551392: step 894, loss 0.0793303, acc 0.99
2018-04-13T00:32:35.152772: step 895, loss 0.0796448, acc 0.97
2018-

2018-04-13T00:35:45.689537: step 1003, loss 0.0580828, acc 0.99
2018-04-13T00:35:47.302094: step 1004, loss 0.0503707, acc 0.99
2018-04-13T00:35:48.903786: step 1005, loss 0.121831, acc 0.97
2018-04-13T00:35:50.515405: step 1006, loss 0.0437205, acc 1
2018-04-13T00:35:52.123909: step 1007, loss 0.0550719, acc 0.99
2018-04-13T00:35:53.741296: step 1008, loss 0.0850687, acc 0.99
2018-04-13T00:35:55.355682: step 1009, loss 0.098847, acc 0.98
2018-04-13T00:35:56.962359: step 1010, loss 0.0644178, acc 0.98
2018-04-13T00:35:58.564542: step 1011, loss 0.0557212, acc 0.99
2018-04-13T00:36:00.171330: step 1012, loss 0.0971138, acc 0.98
2018-04-13T00:36:01.779857: step 1013, loss 0.0621171, acc 0.99
2018-04-13T00:36:03.395736: step 1014, loss 0.0462939, acc 1
2018-04-13T00:36:05.006780: step 1015, loss 0.0736223, acc 0.98
2018-04-13T00:36:06.616934: step 1016, loss 0.148317, acc 0.96
2018-04-13T00:36:08.223182: step 1017, loss 0.0667969, acc 0.99
2018-04-13T00:36:09.838026: step 1018, loss 0.038

2018-04-13T00:39:17.501863: step 1128, loss 0.0362549, acc 1

Test set accuracy:
2018-04-13T00:39:24.730829: step 1128, loss 2.6692, acc 0.135

